In [1]:
import os
import numpy as np
import cv2 as cv
from utils import Utils
from sklearn.mixture import GaussianMixture

In [2]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [3]:

def getThresholdedHand(frame):
    # Convert image to HSV
    hsvim = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # Lower boundary of skin color in HSV
    lower = np.array([0, 48, 80], dtype="uint8")
    # Upper boundary of skin color in HSV
    upper = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv.inRange(hsvim, lower, upper)

    # Gaussian filter (blur) to remove noise
    skinMask = cv.GaussianBlur(skinMask, (17, 17), 0)

    # get thresholded image
    # ret, thresh1 = cv.threshold(
    # skinMask, 100, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    thresh1 = cv.adaptiveThreshold(
        skinMask, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 355, 5)

    return thresh1

In [4]:

# # while True:
# class_dir = os.path.join(menPath, f"{0}")
# imgPath = os.path.join(class_dir, f'{0}_men ({1}).JPG')
# img = cv.imread(imgPath)
# img = getThresholdedHand(img)
# cv.imshow("img", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [5]:
def getMaskedHand2(frame):
        # Convert image to Lab
        rgpImg = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        labImg = cv.cvtColor(rgpImg, cv.COLOR_RGB2LAB)
        # The values of L,A and B channels are  mapped  in  the  range  from  0  up  to  255 
        meanL = np.mean(labImg[:, :, 0])
        meanA = np.mean(labImg[:, :, 1])
        meanB = np.mean(labImg[:, :, 2])
        stdL = np.std(labImg[:, :, 0])
        # stdB = np.std(labImg[:, :, 2])

        if meanA + meanB <=256:
                print("first")
                mask =labImg[:, :, 0] <= ( meanL - stdL/3)
                labImg[mask] = np.array([255, 128, 128])
                labImg[(~mask)] = np.array([0, 128, 128])
        else:
                print("second")
                mask1 = labImg[:, :, 0] <=  ( meanL )
                mask2 = labImg[:,:, 2] <=  (meanB)
                mask = mask1 & mask2
                labImg[mask] = np.array([255, 128, 128])#white
                labImg[(~mask)] = np.array([0, 128, 128])#black
        
        rgpImg2 = cv.cvtColor(labImg, cv.COLOR_LAB2RGB)
        NormalizedImg = cv.normalize(rgpImg2, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)

        kernel = np.ones((13, 13), np.uint8)
        img_dilation = cv.dilate(NormalizedImg, kernel, iterations=1)
        img_erosion = cv.erode(img_dilation, kernel, iterations=1)
        # Gaussian filter (blur) to remove noise
        gaussImg = cv.GaussianBlur(img_erosion, (23, 23), 0)
        # get the white and black pixels 
        white_pixels = (gaussImg == np.array([1, 1, 1])).all(axis=2)
        black_pixels = (gaussImg == np.array([0, 0, 0])).all(axis=2)
        # get the average of the white and black pixels in the original img
        # avgR_shadow = np.mean(rgpImg[:,:,0][white_pixels])
        # avgG_shadow = np.mean(rgpImg[:,:,1][white_pixels])
        # avgB_shadow = np.mean(rgpImg[:,:,2][white_pixels])
        # avgR_without = np.mean(rgpImg[:,:,0][black_pixels])
        # avgG_without = np.mean(rgpImg[:,:,1][black_pixels])
        # avgB_without = np.mean(rgpImg[:,:,2][black_pixels])

        avgR_shadow = 0
        avgG_shadow = 0
        avgB_shadow = 0
        avgR_without = 0
        avgG_without = 0
        avgB_without = 0

        # loop over the image with the kernel
        for i in range(0, rgpImg.shape[0], 40):
                for j in range(0, rgpImg.shape[1], 40):
                        # get the average of the white and black pixels in the original img
                        if gaussImg[i:i+40, j:j+40, 0][white_pixels[i:i+40, j:j+40]].shape[0] >=100:
                                avgR_shadow = np.mean(rgpImg[i:i+40, j:j+40, 0][white_pixels[i:i+40, j:j+40]])
                                avgG_shadow = np.mean(rgpImg[i:i+40, j:j+40, 1][white_pixels[i:i+40, j:j+40]])
                                avgB_shadow = np.mean(rgpImg[i:i+40, j:j+40, 2][white_pixels[i:i+40, j:j+40]])
                                avgR_without = np.mean(rgpImg[i:i+40, j:j+40, 0][black_pixels[i:i+40, j:j+40]])
                                avgG_without = np.mean(rgpImg[i:i+40, j:j+40, 1][black_pixels[i:i+40, j:j+40]])
                                avgB_without = np.mean(rgpImg[i:i+40, j:j+40, 2][black_pixels[i:i+40, j:j+40]])
                                # print("all",rgpImg[i:i+40, j:j+40, 0][white_pixels[i:i+40, j:j+40]].shape)
                                # the ratio between the average of the shadow region and the average of the non-shadow region
                                ratioR =avgR_shadow/avgR_without
                                ratioG = avgG_shadow / avgG_without
                                ratioB = avgB_shadow / avgB_without
                                # print(ratioR, ratioG, ratioB)
                                rgpImg[i:i+40, j:j+40][white_pixels[i:i+40, j:j+40]] = np.array([avgR_shadow, avgG_shadow, avgB_shadow])
                                rgpImg[i:i+40, j:j+40] = cv.medianBlur(rgpImg[i:i+40, j:j+40], 5)
                # replace the shadow pixels with the average of the shadow region
        # rgpImg[white_pixels] = np.array([avgR_shadow, avgG_shadow, avgB_shadow])
        # the ratio between the average of the shadow region and the average of the non-shadow region
        # ratioR =avgR_shadow/avgR_without
        # ratioG = avgG_shadow / avgG_without
        # ratioB = avgB_shadow / avgB_without
        # print(ratioR, ratioG, ratioB)

        # rgpImg[white_pixels]= np.array([ratioR, ratioG, ratioB]) 
        # To eliminate the over-illuminated pixels
        # rgpImg = cv.medianBlur(rgpImg, 17)
        return rgpImg

In [6]:
def maskHand(img):
    # convert the image from BGR to ycrcb
    ycrcbImg = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)
    # get the ycrcb channels
    y, cr, cb = cv.split(ycrcbImg)
    # apply the gaussian filter to remove noise
    gaussImg = cv.GaussianBlur(cr, (5, 5), 0)
    # apply the threshold to get the mask
    _, mask = cv.threshold(gaussImg, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    # apply the dilation to get the mask
    mask = cv.dilate(mask, kernel, iterations=1)
    # apply the erosion to get the mask
    mask = cv.erode(mask, kernel, iterations=1)
    # apply the median filter to remove noise
    mask = cv.medianBlur(mask, 5)
    # apply the closing to fill the holes
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    # apply the opening to remove noise
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    return mask

In [7]:
# Load the image
img = cv.imread('../Dataset_0-5/men/4/4_men (1).JPG')

# Convert the image from BGR to YCbCr
ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

# Extract the Cb and Cr channels from the YCbCr image
cb_channel = ycbcr_image[:, :, 1]
cr_channel = ycbcr_image[:, :, 2]

# Reshape the Cb and Cr channels into 1D arrays
cb_channel_1d = np.reshape(cb_channel, (-1, 1))
cr_channel_1d = np.reshape(cr_channel, (-1, 1))

# Concatenate the Cb and Cr channels into a single 2D array
cbcr_channels = np.concatenate((cb_channel_1d, cr_channel_1d), axis=1)

# Fit a Gaussian mixture model to the Cb and Cr channels
gmm = GaussianMixture(n_components=2)
gmm.fit(cbcr_channels)

# Predict the labels for each pixel in the Cb and Cr channels
labels = gmm.predict(cbcr_channels)

# Reshape the labels back into a 2D array
labels_2d = np.reshape(labels, cb_channel.shape)

# Convert the labels to uint8 data type
labels_2d_uint8 = labels_2d.astype(np.float64)
print(labels_2d_uint8.max())
# Display the labels as an image
cv.imshow('Labels',  cv.resize(labels_2d_uint8, (1024, 512)))
cv.waitKey(0)


-1

In [ ]:
for i in range(50, 60):
    out = maskHand(cv.imread(f"../Dataset_0-5/men/4/4_men ({i}).JPG"))
    cv.imshow(f"shadow{i}",  cv.resize(out, (1024, 512)))
    # getMasked = Utils.getMaskedHand(out)
    # cv.imshow(f"maskedHand{i}", cv.resize(getMasked, (1024, 512)))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [11]:
labels_2d_uint8 = labels_2d.astype(np.float64)
print(labels_2d_uint8.max())
# Display the labels as an image
cv.imshow('Labels',  cv.resize(labels_2d_uint8, (1024, 512)))
cv.waitKey(0)


1.0


-1